In [54]:
import pandas as pd
import numpy as np



In [90]:
# data processing to only view california real estate data
df_sale_counts_by_zip = pd.read_csv('data/raw/Sale_Counts_Zip.csv', encoding="ISO-8859-1")
df_median_all_home_price_by_zip = pd.read_csv('data/raw/Zip_MedianListingPrice_AllHomes.csv',  encoding="ISO-8859-1")
df_all_home_by_zip = pd.read_csv('data/raw/Zip_Zhvi_AllHomes.csv',  encoding="ISO-8859-1")
df_single_family_residence_by_zip = pd.read_csv('data/raw/Zip_Zhvi_SingleFamilyResidence.csv',  encoding="ISO-8859-1")

df_sale_counts_by_zip_california = df_sale_counts_by_zip.loc[df_sale_counts_by_zip['StateName']=='California']
df_all_home_by_zip_california = df_all_home_by_zip.loc[df_all_home_by_zip['State']=='CA']
df_single_family_residence_by_zip_california = df_single_family_residence_by_zip.loc[df_single_family_residence_by_zip['State']=='CA']
df_median_all_home_price_by_zip_california = df_median_all_home_price_by_zip[df_median_all_home_price_by_zip['State']=='CA']


In [91]:
# data processing to load all IPO Data between 1997 and present data. This data has been scraped using code from src/web_scrapers.py
df_full_ipo_date_97_to_19 = pd.read_csv('data/processed/1997-04_2019_full_ipo_data.csv',encoding="ISO-8859-1")



In [176]:
#data processing of census data to create demographics by zip code
zip_census_data = pd.read_csv('data/raw/zip_census_bureau_economic_characteristics_2017.csv',encoding="ISO-8859-1")

In [138]:
# type wrangling of all pandas series in each dataframe
df_sale_counts_by_zip_california["RegionID"] = df_sale_counts_by_zip_california["RegionID"].astype(dtype=np.int64)
df_sale_counts_by_zip_california['StateName'] = df_sale_counts_by_zip_california['StateName'].astype('str')

df_full_ipo_date_97_to_19['Date Filed'] =  pd.to_datetime(df_full_ipo_date_97_to_19['Date Filed'], errors='coerce', format='%m/%d/%Y')
df_full_ipo_date_97_to_19['Lockup Expiration Date'] =  pd.to_datetime(df_full_ipo_date_97_to_19['Lockup Expiration Date'], errors='coerce', format='%m/%d/%Y')


/Users/aaron/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/aaron/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [177]:
# data wrangling of census data

zip_census_data.rename(columns = {'Id2':'Zipcode'}, inplace=True)
zip_census_data.rename(columns = {'Percent; EMPLOYMENT STATUS - Civilian labor force - Unemployment Rate':'Unemployment Rate'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Less than $10,000':'l10000'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $10,000 to $14,999':'l15000'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $15,000 to $24,999':'l25000'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)':'Mean Travel Time to Work Estimate (minutes)'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017| INFLATION-ADJUSTED DOLLARS) - Total households - $200,000 or more':'Percent of Households with Income Greater than $200,000'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Median household income (dollars)':'Median Household Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Mean household income (dollars)':'Mean Household Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)':'Per Capita Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage':'Percent of Population with no Health Insurance Coverage'}, inplace=True)
zip_census_data.rename(columns = {'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people':'Percent of People whose Income in the Past 12 months has been Below Poverty Level'}, inplace=True)


filtered_census_data_columns = ['Zipcode',
                               'Unemployment Rate',
                               'Mean Travel Time to Work Estimate (minutes)',
                               'Percent of Households with Income Greater than $200,000',
                               'Median Household Income Estimate (dollars)',
                               'Mean Household Income Estimate (dollars)',
                               'Per Capita Income Estimate (dollars)',
                               'Percent of Population with no Health Insurance Coverage',
                               'Percent of People whose Income in the Past 12 months has been Below Poverty Level',
                               'Percent of Households With Income Less Than $24,999']

In [178]:
zip_census_data['l10000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l10000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l10000'] = zip_census_data['l10000'].astype(float)

zip_census_data['l15000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l15000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l15000'] = zip_census_data['l15000'].astype(float)

zip_census_data['l25000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l25000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l25000'] = zip_census_data['l25000'].astype(float)
zip_census_data["Percent of Households With Income Less Than $24,999"] = zip_census_data['l10000'] + zip_census_data['l15000']+zip_census_data['l25000']
zip_census_data.filter(filtered_census_data_columns)

,Zipcode,Unemployment Rate,Mean Travel Time to Work Estimate (minutes),Median Household Income Estimate (dollars),Mean Household Income Estimate (dollars),Per Capita Income Estimate (dollars),Percent of Population with no Health Insurance Coverage,Percent of People whose Income in the Past 12 months has been Below Poverty Level,"Percent of Households With Income Less Than $24,999"
0,89010,1.4,10.3,46111,56505,25603,28.9,7.8,20.6
1,89019,12.5,28.1,35856,55862,26622,6.7,16.5,32.3
2,89046,1.9,11.9,31389,48033,29959,20.9,9.3,24.5
3,89060,13.3,28.2,38370,47664,20789,16.4,22.5,30.4
4,89061,2.2,31.3,54375,65354,29760,4.5,5.6,17.9
5,89439,10.5,21.4,56917,82409,37676,15.4,8.1,10.1
6,90001,11.2,32.4,35660,45650,11882,20.7,31.9,34.6
7,90002,12.3,34.9,34000,44463,11807,20.9,33.8,38.4
8,90003,12.5,35.7,34397,43342,11305,21.8,33.1,38.0
9,90004,7.1,32.1,46581,82091,30590,24.1,19.2,24.6


In [197]:
# creating list and dictionary for storing all the zipcodes in silicon valley + san Francisco
import csv
zipcodes = {}
zip_list=[]

with open('data/raw/Santa_Clara_County_Zipcodes.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= ''
    zip_list.append(your_list[z][0])
    
with open('data/raw/San_Mateo_County_Zipcodes.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= ''
    zip_list.append(your_list[z][0])
    
with open('data/raw/San_Francisco_County_Zipcodes.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= ''
    zip_list.append(your_list[z][0])
    
with open('data/raw/Alameda_County_Zipcodes.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= ''
    zip_list.append(your_list[z][0])

len(zip_list)

173

In [221]:
import urllib.request
import json

def calculate_distance_between_zips(zipcode_1,zipcode_2):
    # api-endpoint 
    URL_base = "http://www.zipcodeapi.com/rest/mxftDRxaOd51aP7jf6PQrLAD5dFyaFomjOv7Dj4fkrCsqMb1wC7YTotatqCCT96F/distance.json/"

    URL = URL_base+zipcode_1+'/'+zipcode_2+'/mile'
    # sending get request and saving the response as response object 
    contents = urllib.request.urlopen(URL).read()
  

    # printing the output 
    return json.loads(contents)['distance']